In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ImVisualize import drawLandmarks

In [2]:
imgpath = "Image/me.jpg"
img = cv2.imread(imgpath)
img.shape

(1280, 852, 3)

In [3]:
# cv2.imshow("Image", img)
# cv2.waitKey(1000)
# cv2.destroyAllWindows()

In [4]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_img.shape

(1280, 852)

In [5]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [6]:
face = face_classifier.detectMultiScale(
    img, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40)
)
print(face)

for (x, y, w, h) in face:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 100, 0), 4)

[[161 217 515 515]]


In [7]:
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [8]:
# plt.figure(figsize=(10,5))
# plt.imshow(img_rgb)
# plt.axis('off')

<h1>Real-time Face Detection<h1>

In [9]:
video_capture = cv2.VideoCapture(0)


In [10]:
def face_bounding(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(image=gray_img, scaleFactor=1.1, minNeighbors=3, minSize=(70, 70))

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 155, 0), 4)
    return image

In [11]:
while True:
    result, image = video_capture.read()
    if not result:
        print('Some error with video capturing')
    
    image = face_bounding(image)
    cv2.imshow("Face detection", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [12]:
video_capture.release()

In [13]:
cv2.imshow('a', image)
key = cv2.waitKey(0)
cv2.destroyAllWindows() 
print(key)

113


In [16]:
pointPath = "C:\\Users\\admin\\Downloads\\drive-download-20240314T005400Z-001\\helen\\trainset\\2908549_1.pts"
impath = "C:\\Users\\admin\\Downloads\\drive-download-20240314T005400Z-001\\helen\\trainset\\2908549_1.jpg"


In [17]:
image = drawLandmarks(image_path=impath, points_path=pointPath)
cv2.imshow("LandMark: ", image)
cv2.waitKey(0)
cv2.destroyAllWindows()